In [2]:
# Installing Pathway
!pip install -U pathway

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 6.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 

In [1]:
# Importing data
from google.colab import files
uploaded = files.upload()

Saving input_data.csv to input_data.csv


In [1]:
# Importing Pathway
import pathway as pw
import datetime

In [35]:
import pandas as pd

# Load the raw data
df = pd.read_csv("input_data.csv", header=None, skiprows=1)

# Build timestamp from last two columns (date and time)
df["Timestamp"] = pd.to_datetime(df.iloc[:, -2] + " " + df.iloc[:, -1], dayfirst=True)

# Create a cleaned dataframe with proper headers
df_cleaned = pd.DataFrame({
    "Timestamp": df["Timestamp"],
    "SystemCodeNumber": df.iloc[:, 1],
    "Occupancy": df.iloc[:, 2].astype(int),
    "Capacity": df.iloc[:, 5].astype(int),
    "QueueLength": df.iloc[:, 8].astype(int),
    "TrafficConditionNearby": df.iloc[:, 7],
    "IsSpecialDay": df.iloc[:, 9].astype(int),
    "VehicleType": df.iloc[:, 6]
})
# Save cleaned data
df_cleaned.to_csv("clean_input.csv", index=False)


In [39]:
# Parking Schema
class ParkingSchema(pw.Schema):
    Timestamp: str
    SystemCodeNumber: str
    Occupancy: int
    Capacity: int
    QueueLength: int
    TrafficConditionNearby: str
    IsSpecialDay: int
    VehicleType: str

In [40]:
# Reading Data
data = pw.io.csv.read(
    "clean_input.csv",
    schema=ParkingSchema,
    mode="static"
)

In [41]:
# Adding Hour Column
@pw.udf
def extract_hour(ts: str) -> int:
    return datetime.datetime.fromisoformat(ts).hour

data = data.with_columns(
    Hour=extract_hour(pw.this.Timestamp)
)

In [92]:
import math

@pw.udf
def compute_price(
    occupancy, capacity, queue_length,
    traffic_condition, is_special_day,
    vehicle_type, hour
):
    base_price = 10.0
    min_price = 5.0
    max_price = 20.0

    # Raw demand initialization
    raw_demand = ( occupancy + queue_length ) / max(1, capacity)

    # Contextual demand adjustments (additive)
    if is_special_day:
        raw_demand += 0.2
    if traffic_condition == "High":
        raw_demand -= 0.25
    elif traffic_condition == "Low":
        raw_demand += 0.15
    if vehicle_type == "Truck":
        raw_demand += 0.2
    elif vehicle_type == "Bike":
        raw_demand -= 0.15
    if 11 <= hour <= 14:
        raw_demand += 0.2
    elif 8 <= hour <= 9:
        raw_demand -= 0.15

    # Normalising the demand
    norm_demand = (raw_demand -1.0) / 0.8 - 0.4
    norm_demand = max(-1, min(1, norm_demand))

    # Price Calculation
    lambda_scale = 1.3
    price = base_price * ( 1 + norm_demand )

    # Clamping the price between $5 and $20
    price = round(max(min_price, min(max_price, price)), 2)

    return price

In [93]:
# Output
output_table = data.with_columns(
    Price=compute_price(
        pw.this.Occupancy, pw.this.Capacity, pw.this.QueueLength,
        pw.this.TrafficConditionNearby, pw.this.IsSpecialDay,
        pw.this.VehicleType, pw.this.Hour
    )
)

In [94]:
# Print in the console
pw.debug.compute_and_print(output_table)

            | Timestamp           | SystemCodeNumber | Occupancy | Capacity | QueueLength | TrafficConditionNearby | IsSpecialDay | VehicleType | Hour | Price
^0AKRQFW... | 2016-10-04 07:59:00 | BHMBCCMKT01      | 577       | 61       | 1           | low                    | 0            | car         | 7    | 20.0
^B5Y9HVA... | 2016-10-04 07:59:00 | BHMBCCTHL01      | 387       | 120      | 2           | low                    | 0            | car         | 7    | 20.0
^QWJ0B9C... | 2016-10-04 07:59:00 | BHMEURBRD01      | 470       | 117      | 2           | low                    | 0            | car         | 7    | 20.0
^1E1M7D4... | 2016-10-04 07:59:00 | BHMMBMMBX01      | 687       | 264      | 2           | low                    | 0            | car         | 7    | 20.0
^T858VEQ... | 2016-10-04 07:59:00 | BHMNCPHST01      | 1200      | 237      | 2           | low                    | 0            | bike        | 7    | 20.0
^X90VHXW... | 2016-10-04 07:59:00 | BHMNCPNST01    